In [5]:
!pip install transformers tensorflow pandas numpy scikit-learn

### check the my T4 GPU is working or not


In [6]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    name = tf.config.experimental.get_device_details(gpus[0]).get("device_name", "Unknown")
    print("CUDA available: True")
    print("GPU Name:", name)
else:
    print("CUDA available: False")
    print("GPU: No GPU")

CUDA available: True
GPU Name: Tesla T4


### mount the drive

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


### import the cleand dataset (to the dataframe)

In [9]:
import pandas as pd

df=pd.read_csv("/content/drive/MyDrive/model_datasets/datasets/Trim_data.csv")

In [10]:
df.head()

,lemmatized_text,Sentiment
0,no content general knowledge instructor very h...,0
1,enjoy topic lecture augment break piece partic...,1
2,lecture boring monotonous,0
3,bit complex beginner but learn lot thank,2
4,week truly beginnerintro course stuff entire c...,0


## import the numpy, sklearn(Split the data and test the accuracy), and also tensorflow

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import tensorflow as tf

## download the bert model and for fine tune

In [13]:
from transformers import BertTokenizer, TFAutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3,from_pt=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
model

## Convert the 'lemmatized_text' column to string type, replacing any potential NaN values with an empty string


In [15]:
df['lemmatized_text'] = df['lemmatized_text'].astype(str).fillna('')

##  lemmatized data assign to x and Sentiment assign to y
## split the data for train and validation

In [16]:
X = list(df["lemmatized_text"])
y = list(df["Sentiment"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)

## tokenize the data x train and x val

In [17]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512,return_tensors='tf')
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512,return_tensors='tf')

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


## check the tokernized keys

In [18]:
X_train_tokenized.keys()

KeysView({'input_ids': <tf.Tensor: shape=(9600, 373), dtype=int32, numpy=
array([[  101,  3231,  2025, ...,     0,     0,     0],
       [  101,  2607,  2200, ...,     0,     0,     0],
       [  101,  4366, 10780, ...,     0,     0,     0],
       ...,
       [  101,  2307,   102, ...,     0,     0,     0],
       [  101,  2460, 10480, ...,     0,     0,     0],
       [  101,  2428,  2572, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(9600, 373), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(9600, 373), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]],

## check the count of the x_train and x_val(validation)

In [19]:
len(X_train),len(X_val)

(9600, 2400)

## creates TensorFlow datasets(shuffling and batching)

In [20]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_tokenized),
    y_train
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_val_tokenized),
    y_val
)).batch(16)

## creates training and validation datasets using your tokenized input and coresponding labels

In [22]:
import tensorflow as tf

# Example: X_train_tokenized is a dictionary with 'input_ids' and 'attention_mask'
def create_tf_dataset(tokenized_inputs, labels, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': tokenized_inputs['input_ids'],
            'attention_mask': tokenized_inputs['attention_mask']
        },
        labels
    ))
    return dataset.batch(batch_size)


In [23]:
train_dataset = create_tf_dataset(X_train_tokenized, y_train)
val_dataset = create_tf_dataset(X_val_tokenized, y_val)

## Configure Training

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

## Train the Model

In [26]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

Epoch 1/3
600/600 [==============================] - 837s 1s/step - loss: 0.7712 - accuracy: 0.6426 - val_loss: 0.7213 - val_accuracy: 0.6671
Epoch 2/3
600/600 [==============================] - 783s 1s/step - loss: 0.6262 - accuracy: 0.7264 - val_loss: 0.7393 - val_accuracy: 0.6779
Epoch 3/3
600/600 [==============================] - 783s 1s/step - loss: 0.4846 - accuracy: 0.8065 - val_loss: 0.8108 - val_accuracy: 0.6542


## eveluvate with matrix(validate the data)

In [27]:
def compute_metrics(y_true, y_pred):
    y_pred = np.argmax(y_pred, axis=1)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average='weighted'),
        "recall": recall_score(y_true, y_pred, average='weighted'),
        "f1": f1_score(y_true, y_pred, average='weighted')
    }

# Predict on validation set
val_preds = model.predict(val_dataset).logits
metrics = compute_metrics(y_val, val_preds)
print(metrics)

150/150 [==============================] - 56s 352ms/step
{'accuracy': 0.6541666666666667, 'precision': 0.6779181641198374, 'recall': 0.6541666666666667, 'f1': 0.6606211338208695}


## save the full model to use after train

In [28]:

save_dir = "/content/drive/MyDrive/model_datasets/datasets/model_trained"

# Save model (TensorFlow format)
model.save_pretrained(save_dir)

# Save tokenizer (same for both frameworks)
tokenizer.save_pretrained(save_dir)

# Verify files exist
!ls -lh {save_dir}

total 419M
-rw------- 1 root root  764 Jul  7 16:29 config.json
-rw------- 1 root root  125 Jul  7 16:29 special_tokens_map.json
-rw------- 1 root root 418M Jul  7 16:29 tf_model.h5
-rw------- 1 root root 1.3K Jul  7 16:29 tokenizer_config.json
-rw------- 1 root root 227K Jul  7 16:29 vocab.txt


## check the working properly or not

In [31]:
import numpy as np

# Sample text for inference
text = "its ok"

# Tokenize with TensorFlow tensors
inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='tf')

# Get model predictions
outputs = model(inputs)
logits = outputs.logits

# Convert to probabilities using TensorFlow
probabilities = tf.nn.softmax(logits, axis=-1).numpy()[0]

print("Raw probabilities:", probabilities)

# Get predicted class
predicted_class = np.argmax(probabilities)
print("Predicted class index:", predicted_class)

# Map to sentiment labels
sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}
sentiment = sentiment_labels.get(predicted_class, "unknown")

print(f"The sentiment of the text is: {sentiment}")

Raw probabilities: [0.29059088 0.68345886 0.02595028]
Predicted class index: 1
The sentiment of the text is: neutral
